In [ ]:
# ai_art_classification/
# ├── train/
# │   ├── AI_GENERATED/         ← fake → label = 1
# │   └── NON_AI_GENERATED/     ← real → label = 0
# ├── test/

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!unzip /content/drive/MyDrive/ai_art_classification.zip -d /content/

Streaming output truncated to the last 5000 lines.
  inflating: /content/ai_art_classification/train/NON_AI_GENERATED/8898.jpg  
  inflating: /content/__MACOSX/ai_art_classification/train/NON_AI_GENERATED/._8898.jpg  
  inflating: /content/ai_art_classification/train/NON_AI_GENERATED/496.jpg  
  inflating: /content/__MACOSX/ai_art_classification/train/NON_AI_GENERATED/._496.jpg  
  inflating: /content/ai_art_classification/train/NON_AI_GENERATED/1102.jpg  
  inflating: /content/__MACOSX/ai_art_classification/train/NON_AI_GENERATED/._1102.jpg  
  inflating: /content/ai_art_classification/train/NON_AI_GENERATED/3073.jpg  
  inflating: /content/__MACOSX/ai_art_classification/train/NON_AI_GENERATED/._3073.jpg  
  inflating: /content/ai_art_classification/train/NON_AI_GENERATED/18276.jpg  
  inflating: /content/__MACOSX/ai_art_classification/train/NON_AI_GENERATED/._18276.jpg  
  inflating: /content/ai_art_classification/train/NON_AI_GENERATED/14464.jpg  
  inflating: /content/__MACOSX/ai_a

In [3]:
import os

base_dir = "/content/ai_art_classification"
print("Train folders:", os.listdir(os.path.join(base_dir, "train")))

Train folders: ['NON_AI_GENERATED', 'AI_GENERATED']


In [4]:
# Extract Features with DINOv2

import os
from PIL import Image
import torch
from transformers import AutoImageProcessor, AutoModel
from tqdm import tqdm

# Load DINOv2
processor = AutoImageProcessor.from_pretrained("facebook/dinov2-base")
model = AutoModel.from_pretrained("facebook/dinov2-base")
model.eval()

def extract_dino_features(img_path):
    image = Image.open(img_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        cls_embedding = outputs.last_hidden_state[:, 0, :]  # [1, 768]
    return cls_embedding.squeeze().numpy()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/436 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


config.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [5]:
# Build feature dataset

import numpy as np

# Set data path
data_dir = "ai_art_classification/train"
label_map = {
    "NON_AI_GENERATED": 0,
    "AI_GENERATED": 1}

X, y = [], []

for class_name, label in label_map.items():
    class_path = os.path.join(data_dir, class_name)
    for img_file in tqdm(os.listdir(class_path), desc=f"Processing {class_name}"):
        img_path = os.path.join(class_path, img_file)
        try:
            img = Image.open(img_path).convert("RGB")
            feat = extract_dino_features(img_path)
            X.append(feat)
            y.append(label)
        except Exception as e:
            print(f"Failed to process {img_file}: {e}")

X = np.array(X)
y = np.array(y)

# Save to disk
os.makedirs("features", exist_ok=True)
np.savez_compressed("features/dino_features_train.npz", X=X, y=y)
print("✅ Features saved to features/dino_features_train.npz")

Processing AI_GENERATED: 100%|██████████| 10330/10330 [13:22<00:00, 12.87it/s]


✅ Features saved to features/dino_features_train.npz


In [2]:
!cp /content/features/dino_features_train.npz /content/drive/MyDrive/

In [4]:
!ls -lh /content/features

total 51M
-rw-r--r-- 1 root root 51M May 16 02:34 dino_features_train.npz


In [6]:
import numpy as np

data = np.load("features/dino_features_train.npz")
X = data["X"]
y = data["y"]
print("Loaded features:", X.shape, y.shape)

Loaded features: (18618, 768) (18618,)


In [9]:
# train classifier

import os
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix
import joblib

# Split and train
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

clf = LinearSVC(dual=False, max_iter=5000)
clf.fit(X_train, y_train)

# Save model
os.makedirs("model", exist_ok=True)
joblib.dump(clf, "model/svm_model.pkl")
print("✅ Model saved to model/svm_model.pkl")

✅ Model saved to model/svm_model.pkl


In [10]:
# Predict on the test set
y_pred = clf.predict(X_test)

# Accuracy
accuracy = (y_pred == y_test).mean()
print(f"Accuracy: {accuracy * 100:.2f}%")

# Detailed report
print("\n Classification Report:")
print(classification_report(y_test, y_pred, target_names=["Human-made", "AI-generated"]))

# Confusion matrix
print("\n Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Accuracy: 90.82%

 Classification Report:
              precision    recall  f1-score   support

  Human-made       0.91      0.88      0.90      1658
AI-generated       0.91      0.93      0.92      2066

    accuracy                           0.91      3724
   macro avg       0.91      0.91      0.91      3724
weighted avg       0.91      0.91      0.91      3724


 Confusion Matrix:
[[1460  198]
 [ 144 1922]]


### Streamlit app web demo

In [ ]:
# your_project/
# ├── app.py                  ← Streamlit app
# ├── model/
# │   └── svm_model.pkl       ← Trained classifier
# ├── utils/
# │   └── dino_features.py    ← Feature extractor

In [ ]:
#  dino_features.py (feature extraction)

# utils/dino_features.py
import torch
from PIL import Image
from transformers import AutoImageProcessor, AutoModel
import numpy as np

# Load DINOv2 base once
processor = AutoImageProcessor.from_pretrained("facebook/dinov2-base")
model = AutoModel.from_pretrained("facebook/dinov2-base")
model.eval()

def extract_dino_feature(image: Image.Image) -> np.ndarray:
    inputs = processor(images=image, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        cls_token = outputs.last_hidden_state[:, 0, :]  # [1, 768]
    return cls_token.squeeze().numpy()

In [ ]:
#  app.py (Streamlit UI)

# app.py

import streamlit as st
import joblib
from PIL import Image
from utils.dino_features import extract_dino_feature
import numpy as np
import matplotlib.pyplot as plt

# Page title
st.set_page_config(page_title="AI Art Fake Detection", layout="centered")
st.title("🎨 AI Art Fake Detection App")
st.write("Upload an artwork to predict whether it's **human-made** or **AI-generated** using DINOv2 features + SVM classifier.")

# Upload image
uploaded_file = st.file_uploader("Upload Image", type=["jpg", "jpeg", "png"])

# Prediction logic
if uploaded_file:
    # Show image
    image = Image.open(uploaded_file).convert("RGB")
    st.image(image, caption="Uploaded Artwork", use_column_width=True)

    st.info("🔍 Extracting features using DINOv2...")
    features = extract_dino_feature(image).reshape(1, -1)

    # Load trained model
    clf = joblib.load("model/svm_model.pkl")

    # Prediction
    prob = clf.predict_proba(features)[0]
    pred = clf.predict(features)[0]

    # Output result
    label = "🧠 Human-made" if pred == 0 else "🤖 AI-generated"
    st.subheader(f"Prediction: {label}")
    st.write(f"Confidence: {prob[pred]*100:.2f}%")

    # Confidence bar chart
    st.write("### 🔎 Prediction Confidence")
    classes = ["Human-made", "AI-generated"]
    colors = ["#4CAF50", "#FF5722"]

    fig, ax = plt.subplots()
    bars = ax.bar(classes, prob * 100, color=colors)
    ax.set_ylim([0, 100])
    ax.set_ylabel("Confidence (%)")
    ax.set_title("Prediction Confidence")

    # Annotate bars
    for bar, p in zip(bars, prob):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width() / 2.0, height + 1, f"{p*100:.1f}%", ha='center')

    st.pyplot(fig)

# Footer
st.markdown("---")
st.caption("Built using DINOv2, Streamlit, and scikit-learn.")

In [ ]:
!streamlit run app.py